In [1]:
# import libraries
import requests
import re
import pandas as pd
import numpy as np
import datetime as dt
from math import sin, cos, sqrt, atan2, radians

In [2]:
# create start and end dates
start_date = '2019-05-01'
end_date = dt.datetime.today() - dt.timedelta(days = 1)
date_range = pd.date_range(start_date, end_date)

In [5]:
# create URL parameters for API call
payload = {'start': start_date, 'end': end_date.strftime('%Y-%m-%d'), 'type':'daily', 'stns': 'ALL'}

In [6]:
# make API call and store in variable knmi
knmi = requests.get('http://projects.knmi.nl/klimatologie/daggegevens/getdata_dag.cgi', params = payload)

In [11]:
# split text response in separate lines
knmi_strings = re.split('\r\n', knmi.text)

In [54]:
# check results
knmi_strings[:100]

['# BRON: KONINKLIJK NEDERLANDS METEOROLOGISCH INSTITUUT (KNMI)',
 '# Opmerking: door stationsverplaatsingen en veranderingen in waarneemmethodieken zijn deze tijdreeksen van dagwaarden mogelijk inhomogeen! Dat betekent dat deze reeks van gemeten waarden niet geschikt is voor trendanalyse. Voor studies naar klimaatverandering verwijzen we naar de gehomogeniseerde reeks maandtemperaturen van De Bilt <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/gehomogeniseerde-reeks-maandtemperaturen-de-bilt> of de Centraal Nederland Temperatuur <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/centraal-nederland-temperatuur-cnt>.',
 '# ',
 '# ',
 '# STN      LON(east)   LAT(north)     ALT(m)  NAME',
 '# 391:         6.197       51.498      19.50  ARCEN',
 '# 370:         5.377       51.451      22.60  EINDHOVEN',
 '# 331:         4.193       51.480       0.00  THOLEN',
 '# 315:         3.998       51.447       0.00  HANSWEERT',
 '# 324:         4.006       51.596       0.00  STAVENISSE',
 

In [13]:
# extract KNMI weather data with regex from text
knmi_weather_data = []

for b in knmi_strings: # search in splitted knmi_strings
    data = re.search('^\s+[0-9]{3},20[0-9]{6}.*', b) # search for strings matching regex
    if data is not None:
        data = data[0] # get first result
        data = re.sub('\s+', '', data) # replace all whitespaces with nothing
        data = re.split(',', data)
        knmi_weather_data.append(data) # append to list

In [56]:
# check result
knmi_weather_data[:3]

[['391',
  '20190601',
  '219',
  '9',
  '12',
  '20',
  '6',
  '0',
  '19',
  '90',
  '13',
  '199',
  '107',
  '4',
  '278',
  '16',
  '94',
  '6',
  '115',
  '71',
  '2567',
  '0',
  '0',
  '0',
  '1',
  '47',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '69',
  '98',
  '4',
  '40',
  '17'],
 ['391',
  '20190602',
  '213',
  '22',
  '25',
  '50',
  '13',
  '0',
  '1',
  '100',
  '12',
  '240',
  '128',
  '4',
  '322',
  '15',
  '113',
  '6',
  '129',
  '79',
  '2697',
  '0',
  '0',
  '0',
  '1',
  '52',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '55',
  '99',
  '4',
  '26',
  '14'],
 ['391',
  '20190603',
  '269',
  '9',
  '15',
  '30',
  '3',
  '0',
  '1',
  '70',
  '3',
  '189',
  '129',
  '23',
  '239',
  '13',
  '119',
  '24',
  '35',
  '21',
  '1502',
  '8',
  '6',
  '3',
  '2',
  '27',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '75',
  '96',
  '4',
  '56',
  '13']]

In [15]:
# put data in dataframe
knmi_weather_df = pd.DataFrame(knmi_weather_data, columns = ['STN','YYYYMMDD','DDVEC','FHVEC','FG','FHX','FHXH','FHN','FHNH', 'FXX', 'FXXH', 'TG' , 'TN', 'TNH', 'TX', 'TXH', 'T10N', 'T10NH', 'SQ', 'SP', 'Q', 'DR', 'RH', 'RHX', 'RHXH', 'EV24', 'PG', 'PX', 'PXH',  'PN', 'PNH', 'VVN', 'VVNH',  'VVX', 'VVXH',  'NG',  'UG', 'UX', 'UXH', 'UN', 'UNH'])

In [16]:
# replace empty values in dataframe with np.nan
knmi_weather_df[knmi_weather_df == ''] = np.nan

In [17]:
# drop unnecessary columns
knmi_weather_df = knmi_weather_df[['STN','YYYYMMDD','TG', 'DR', 'RH', 'SQ']]

# rename column names for better understanding of columns
knmi_weather_df.rename({'STN':'station_id','YYYYMMDD':'date', 'TG': 'mean_temp', 'DR': 'percipation_duration', 'RH': 'percipation_amount', 'SQ': 'sunshine_duration'}, inplace=True, axis=1)

In [18]:
# drop all rows and columns with NaN values
knmi_weather_df = knmi_weather_df.dropna()

In [19]:
# convert weather data back to integers
knmi_weather_df.iloc[:, 2:] = knmi_weather_df.iloc[:, 2:].astype(int)

In [20]:
# fill values lower than zero with zero
knmi_weather_df[knmi_weather_df['percipation_amount'] < 0] = 0
knmi_weather_df[knmi_weather_df['sunshine_duration'] < 0] = 0

In [21]:
# change data type to date for column 'date'
knmi_weather_df['date'] = pd.to_datetime(knmi_weather_df['date'])

In [22]:
# extract KNMI station locations from API response
knmi_locations = []

for i in knmi_strings:
    found = re.search('^\# [0-9]{3}\:.*', i)
    if found is not None:
        knmi_locations.append(found[0])

In [23]:
# extract KNMI station locations from API response - step 2
knmi_coordinates = []
knmi_places = []

for a in knmi_locations:
    first_part = a[:44]
    a = first_part.replace('# ', '')
    a = a.replace(':', '')
    a = re.sub('\s+', ',', a)
    a = re.split(',', a)
    knmi_coordinates.append(a)
    
for b in knmi_locations:
    b = b[46:]
    knmi_places.append(b)

In [24]:
# put KNMI station data in dataframe 
knmi_coordinates = pd.DataFrame(knmi_coordinates, columns = ['station_id', 'latitude', 'longitude', 'elevation'], dtype = float)
knmi_places = pd.DataFrame(knmi_places, columns = ['city'])
knmi_locations = pd.concat([knmi_coordinates, knmi_places], axis = 1)
knmi_locations['station_id'] = knmi_locations['station_id'].astype(object)

In [57]:
# check result
knmi_locations.head(10)

,station_id,latitude,longitude,elevation,city
0,391,6.197,51.498,19.5,ARCEN
1,370,5.377,51.451,22.6,EINDHOVEN
2,331,4.193,51.480,0.0,THOLEN
3,315,3.998,51.447,0.0,HANSWEERT
4,324,4.006,51.596,0.0,STAVENISSE
5,375,5.707,51.659,22.0,VOLKEL
6,380,5.762,50.906,114.3,MAASTRICHT
7,240,4.790,52.318,-3.3,SCHIPHOL
8,286,7.150,53.196,-0.2,NIEUW BEERTA
9,310,3.596,51.442,8.0,VLISSINGEN


In [26]:
# create four random store locations with store id's
store_locations = {101: [51.442070, 5.475730], 102: [53.217600, 6.565110], 103: [52.373460, 4.892750], 104: [52.091270, 5.117820]}

In [27]:
# put store locations with longitudes and latitudes in dataframe
store_locations_df = pd.DataFrame.from_dict(store_locations, orient = 'index', columns = ['longitude', 'latitude'])

In [28]:
# find all distances between all KNMI weather stations and all stores and append to 
# distance_list
distance_list = []

# approximate radius of earth in km
R = 6373.0

for index_a, row_a in store_locations_df.iterrows():
    for index_b, row_b in knmi_locations.iterrows():
        lat1 = radians(row_a['latitude'])
        lon1 = radians(row_a['longitude'])
        lat2 = radians(row_b['latitude'])
        lon2 = radians(row_b['longitude'])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        combined = [index_a, row_b['station_id'], row_b['city'], distance]
        distance_list.append(combined)

In [29]:
# put distance_list in dataframe
distance_df = pd.DataFrame(distance_list, columns = ['store_id', 'station_id', 'city', 'distance'])

In [30]:
# find closest KNMI station per store
distance_df = distance_df.groupby('store_id', as_index=False)['store_id', 'station_id', 'city', 'distance'].apply(lambda x: x.nsmallest(1, 'distance', keep='first'))

In [32]:
# generate random sales data per store
sales_data_list = []

for each in store_locations.keys():
    for other in date_range:
        b = str(round(np.random.normal(loc = 1500, scale = 150),2))
        other = other.strftime('%Y-%m-%d')
        combined = [each, other, b]
        sales_data_list.append(combined)

In [58]:
sales_data_list[:10]

[[101, '2019-05-01', '932.03'],
 [101, '2019-05-02', '2761.66'],
 [101, '2019-05-03', '1155.03'],
 [101, '2019-05-04', '1491.35'],
 [101, '2019-05-05', '-96.73'],
 [101, '2019-05-06', '-633.92'],
 [101, '2019-05-07', '1448.41'],
 [101, '2019-05-08', '3454.48'],
 [101, '2019-05-09', '1050.15'],
 [101, '2019-05-10', '2500.79']]

In [34]:
# put sales data in pandas dataframe
sales_data_df = pd.DataFrame(sales_data_list, columns = ['store_id', 'date', 'amount'])

In [35]:
# change data types for columns 'date' and 'amount'
sales_data_df['date'] = pd.to_datetime(sales_data_df['date'])
sales_data_df['amount'] = sales_data_df['amount'].astype(float)

In [37]:
# merge sales_data_df and distance_df together
df2 = pd.merge(sales_data_df, distance_df, left_on = 'store_id', right_on = 'store_id')

In [44]:
# change data type for station id to int for joining
knmi_weather_df['station_id'] = knmi_weather_df['station_id'].astype(int)

In [45]:
# create final dataframe with all data
df = pd.merge(df2, knmi_weather_df, left_on = ['station_id', 'date'], right_on = ['station_id', 'date'])

In [52]:
# change weather data to integers for calculation of correlations
df.iloc[:,6:] = df.iloc[:,6:].astype(int)

In [53]:
# calculate correlations between sales amount and other columns in dataframe
df.corrwith(df['amount'])

store_id                0.083821
amount                  1.000000
distance               -0.072443
mean_temp               0.007115
percipation_duration   -0.035216
percipation_amount     -0.040988
sunshine_duration      -0.018312
dtype: float64

A strong positive correlation between sales amounts of these virtual fashion retailer stores and mean_temp and sunshine_duration is expected. People tend to go out more with better weather. A negative correlation is expected for sales amount and percipation_duration and percipation_amount. 

Store managers can decide to plan in less worker hours or to not replace sick employees when rain is expected. On the other hand, when weather is expected to improve, more employee hours can be planned in! Profitability of stores can therefore be improved.

<b>Unfortunately, with the randomly generated sales data such correlations can not be found.</b>